Quantum demo, by David Finder.

Howdy, welcome to the quantum demo. We're going to work with _baby_ RSA and _baby_ Shor's algorithm.


First, we're going to cover RSA.

RSA is an asymmetric encryption scheme commonly used for web traffic. Asymmetric means that we can have separate public and private keys. Public key is used for encryption, private key is used for decryption. 


It relies on finding $m^{ed}\equiv m\mod n$ such that e,d, and m are large positive numbers. M is our message, e is our public key, and d is our private key. 
 

First we need to find our public and private key. 

To do this we need to find two very large prime numbers....

In [2]:
p = 13
q = 23

For some definition of very large. Maybe this is a graph theory class, where 7 is a large number.

In [3]:
n=p*q

N is our modulus, this is released publically. 
Next, we calculuate $\lambda(n)$, which is the Carmichael Lambda function. 

$\lambda(n) = lcm(\lambda(p),\lambda(q))$.

Given that p,q are prime, $\lambda(p),\lambda(q) = p-1,q-1$

So we take the lcm(12,22), and keep this secret. 

In [6]:
import math
l = math.lcm(p-1,q-1)
l

132

Next, we choose an integer e, less than $\lambda(n)$, such that the two are coprime. 

In [10]:
from sympy import factorint
factorint(132)
e=5

5 looks like a good value for e. 

Next we determine $d\equiv e^{-1}\mod\lambda(n)$


In [16]:
d=[x for x in range(0,132) if (x*e)%l==1][0]
d

53

Note: Never make your d this small in an actual implementation. Having a small d makes you vulnerable to [Wiener's attack.](https://en.wikipedia.org/wiki/Wiener%27s_attack). Now that we're done snickering:

Our public key is (n,e), and our private key is d. 

We can discard p,q, and $\lambda(n)$

In [22]:
public = (n,e)
private = d
print(public)
print(private)

(299, 5)
53


So give me a message using ASCII characters:

In [31]:
message = input("Please give me a plaintext:")

Please give me a plaintext:lmaom2k


Because our n is so small, we're going to use an ascii encoding, and encrypt each individual value. 

In [32]:
ascii_values = list(map(lambda x: ord(x),message))

In [33]:
ascii_values

[108, 109, 97, 111, 109, 50, 107]

And now to encrypt, we take each of these integers, and take: $c=m^e \mod(n)$

In [42]:
cypher_text = list(map(lambda x: (x**e)%n, ascii_values))
print("".join(list(map(lambda x: chr(x),cypher_text))))
cypher_text


K,,¿


[75, 44, 158, 11, 44, 150, 191]

beautiful

Now to decrypt, we do the same thing again, with our D private key.

In [44]:
uncypher_text = list(map(lambda x: (x**d)%n, cypher_text))
print(uncypher_text)
print("".join(list(map(lambda x: chr(x),uncypher_text))))

[108, 109, 97, 111, 109, 50, 107]
lmaom2k


The magic here stems from the fact that working in addition mod $\lambda (n)$, is equivalent to working in multiplication mod n. 

In particular: $m^{ed} = m^{1+h*\lambda(n)} \mod n$ for some h, because $ed=1 \mod \lambda(n)$ 

We know from Carmichael theorem, that $m^{\lambda(n)}\equiv1$ mod n, which gives us $m^{ed} = m*(e^{\lambda(n)})^h \equiv m*1^h \equiv m$



This is gives us a small problem though.... If we can factor n, which is public, we can find p,q, thus l, and then $\lambda(L)$, which allows us to find d from e by Chinese Remainder theorem. 

Fortunately, factoring numbers is computationally hard. 

The largest we've been able to factor is [RSA-250](https://web.archive.org/web/20200228234716/https://lists.gforge.inria.fr/pipermail/cado-nfs-discuss/2020-February/001166.html)

Which took about 2700 core years for a number with 829 bits. The researchers estimated that a 1024 RSA modulus would take about 500 times as long.[Source](https://eprint.iacr.org/2010/006.pdf). 

These days, 2048 bit modulus are needed. 
Factoring numbers takes sub-exponentially longer with the more bits you add, so with classical computing, we would be in a losing battle against a larger key size. 

Enter quantum computing and shors algorithm. 

In [48]:
from braket.devices import LocalSimulator
from braket.aws import AwsDevice
from braket.experimental.algorithms.shors.shors import (
    shors_algorithm,
    run_shors_algorithm,
    get_factors_from_results
)


ModuleNotFoundError: No module named 'braket.experimental'